# Пару слов о блокноте

__Задача блокнота__: обработать входные данные (привести их к формату, удобному для обучения модели машинного обучения).

__Результат блокнота__: один файл со всеми обработанными данными.

__P. S.__: блокнот является первой частью производственной практики по учебному курсу дополнительного образования. Вторая часть подразумевает использование полученного файла для обучения модели машинного обучения. 

# Предобработка данных

In [1]:
import pandas as pd
import os
import re
import shutil

Единственные переменные, которые можно менять:

In [2]:
path_to_archive = "C:\\Users\\DNS\\Downloads\\all_2908.zip"
destionation_path = "C:\\Users\\DNS\\Documents\\Alexander_Frolov\\Programming_Projects\\ML_annotation_practice\\data"

`path_to_archive` впоследствии был удален из директории для экономии пространства. Запустить код с его использованием не получится

### Извлечение всех файлов

In [3]:
# unpack all the new annotated files
def extract_all(path_to_archive, destionation_path):
    shutil.unpack_archive(path_to_archive, destionation_path)

    path_to_main_folder = os.path.join(destionation_path, 'all_2908')
    for archive in os.listdir(path_to_main_folder):
        shutil.unpack_archive(os.path.join(path_to_main_folder, archive), os.path.join(path_to_main_folder, archive[:-4]))
        os.remove(os.path.join(path_to_main_folder, archive))
    
    for folder in os.listdir(path_to_main_folder):
        shutil.move(os.path.join(path_to_main_folder, folder), destionation_path)
    shutil.rmtree(path_to_main_folder)

In [ ]:
extract_all(path_to_archive, destionation_path)

In [9]:
# read all filepath to a list
ALL_PATHS = []

for folder in os.listdir(destionation_path):
    for file in os.listdir(os.path.join(destionation_path, folder)):
        path_to_file = os.path.join(destionation_path, folder, file)

        ALL_PATHS.append(path_to_file)

print(f"Number of files: {len(ALL_PATHS)}")
print(f"Example: {ALL_PATHS[0]}")

Number of files: 62
Example: C:\Users\DNS\Documents\Alexander_Frolov\Programming_Projects\ML_annotation_practice\data\RPD[FKTI]\edu_vasilev.ya.jsonl


In [10]:
# read additional files
additional_filepath_1 = "C:\\Users\\DNS\\Documents\\Alexander_Frolov\\Programming_Projects\\ML_annotation_practice\\" + \
                        "\\data_additional\\all_extended_proc_annot.jsonl"
additional_filepath_2 = "C:\\Users\\DNS\\Documents\\Alexander_Frolov\\Programming_Projects\\ML_annotation_practice\\" + \
                        "\\data_additional\\all_extended_proc_annot_squeezed.jsonl"
ALL_PATHS.append(additional_filepath_1)
ALL_PATHS.append(additional_filepath_2)

In [11]:
# check example of new file
df = pd.read_json(ALL_PATHS[0], lines=True)
df.head(5)

,id,text,cats,entities,Comments
0,12668,276626\nМатематические основания информатики\n...,[],"[[641, 691, Knowledge], [733, 782, Knowledge],...",[]
1,12672,278248\nКомпьютерная графика\nВ курсе изучаютс...,[],"[[254, 302, Knowledge], [324, 374, Knowledge],...",[]
2,12673,288500\nИнформационная безопасность цифровых т...,[],"[[792, 822, Knowledge], [1321, 1347, Knowledge...",[]
3,12674,282060\nРазработка безопасного программного об...,[],"[[873, 918, Knowledge], [919, 968, Knowledge],...",[]
4,12686,275487\nАвтоматизация функционально-логическог...,[],"[[643, 676, Knowledge], [759, 794, Knowledge],...",[]


In [12]:
# check example of old file
df = pd.read_json(ALL_PATHS[-1], lines=True)
df.head(5)

,id,text,cats,entities,Comments,kind,annot,meta
0,67,Разработчик Flutter Developer. Чем необходимо ...,[Мобильный разработчик],"[[12, 19, Tool], [68, 125, Technology], [128, ...",[],vac,"{'username': 'serov.ilya.2903', 'confirmed_at'...",NaN
1,68,"Задачи, которыми предстоит заниматься:\n \n \n...",[Аналитик],"[[182, 194, Method], [436, 459, SoftSkills], [...",[],vac,"{'username': 'samsonik2012', 'confirmed_at': '...",NaN
2,71,Разработчик Java (Устройства Самообслуживания)...,[],"[[12, 16, ProgLanguage], [258, 270, Technology...",[],vac,"{'username': 'samsonik2012', 'confirmed_at': '...",NaN
3,72,Требования: Знание платформы 1С 8.3.х. Пониман...,[],"[[29, 37, Technology], [49, 77, Technology], [...",[],vac,"{'username': 'samsonik2012', 'confirmed_at': '...",NaN
4,115,Инженер - Программист Delphi. Обязанности: соп...,[],"[[22, 28, ProgLanguage], [43, 56, Method], [59...",[],vac,"{'username': 'muuuuusha', 'confirmed_at': '202...",NaN


### Описание основных функций для обработки

Ниже описаны две функции для извлечения токенов из текста. Вторая функция была написана для теста и, как оказалось, ее токены лучше подходят для трансформера.

In [13]:
# first variation: tokens will be either words or numbers 
def text_to_tokens_1(text):
    text = re.sub("[^А-ЯЁA-Zа-яёa-z0-9\s]", ' ', text)
    text = re.sub("\s+", ' ', text)
    tokens = text.strip().split()
    return tokens 

# seconds variation: tokens will be either words or numbers or the following characters: ":;,.!?()-"
def text_to_tokens_2(text):
    text = text.replace('\n', ' ')
    text = re.sub('[^А-ЯЁA-Zа-яёa-z0-9\s]«»\'":;,.!?()\-', ' ', text)

    i = 0
    # separate words and characters like ":;,.!?()-" by spaces
    while i < len(text):
        if text[i] in "«»\"':;,.!?()-":
            if (i == 0) or (text[i - 1] == ' '):
                # character at the start
                text = text[:i] + text[i] + ' ' + text[i + 1:]
                i += 1
            elif (i == len(text) - 1) or (text[i + 1] == ' '):
                # character at the end
                text = text[:i] + ' ' + text[i] + text[i + 1:]
            else:
                # character in the middle
                text = text[:i] + ' ' + text[i] + ' ' + text[i + 1:]
                i += 2
        else:
            i += 1

    text = re.sub('\s+', ' ', text)
    tokens = text.strip().split()
    return tokens

Ниже описана вспомогательная функция для получения границ всех токенов внутри исходного текста. 

In [14]:
# Returns list of [beginning of the token, end of the token] for each token
def get_bounds_of_tokens_in_text(text, tokens):
    i = 0; tokens_ind = 0; letter_ind = 0
    buffer = ''
    result = []

    while i < len(text):
        if (len(tokens[tokens_ind]) > 0) and (text[i] == tokens[tokens_ind][letter_ind]):
            buffer += text[i]
            letter_ind += 1
            if letter_ind == len(tokens[tokens_ind]):
                result.append([i - len(buffer) + 1, i])
                tokens_ind += 1
                letter_ind = 0
                buffer = ''
                
                if tokens_ind == len(tokens):
                    break
                continue
        else:
            buffer = ''
            letter_ind = 0
        i += 1

    return result

# example
text = "hello, i'm a writer. nice to meet you"
tokens = text_to_tokens_2(text)
print(f"Text: {text}")
print(f"Tokens: {tokens}")
print(f"Result: {get_bounds_of_tokens_in_text(text, tokens)}")

Text: hello, i'm a writer. nice to meet you
Tokens: ['hello', ',', 'i', "'", 'm', 'a', 'writer', '.', 'nice', 'to', 'meet', 'you']
Result: [[0, 4], [5, 5], [7, 7], [8, 8], [9, 9], [11, 11], [13, 18], [19, 19], [21, 24], [26, 27], [29, 32], [34, 36]]


Ниже описана вспомогательная функция - она создает новый массив сущностей, где вместо первых двух элементов (границ сущности внутри текста) я ставлю два индекса - границы сущности внутри массива токенов. <br>
Функция будет работать корректно, если не происходит такого, что символ из текста относится сразу к двум сущностям - в противном случае функция может зациклиться. <br>
Если вдруг понадобится обрабатывать и такие случаи, то следует добавить в функцию сдвиг переменной i обратно после каждого считывания.

In [15]:
# Returns: new array like [begin_ind, end_ind, entity_name] but first two elements are indexes of tokens
# (orirginal entities array have indexes of characters whithin the original text)
# !!! method will work correct if there is no different entities wrapping same words !!!
def get_updated_entites_array(original_text, text_tokens, original_entities):
    new_entities = [[0, 0, ''] for _ in range(len(original_entities))]
    
    token_bounds = get_bounds_of_tokens_in_text(original_text, text_tokens)
    i = 0
    token_ind = -1 # index of the last token encountered
    token_bounds_ind = 0 # current index in token_bounds
    entities_ind = 0 # current index in original_entities
    
    while (i < len(original_text)) and (entities_ind < len(original_entities)):
        if token_bounds_ind < len(token_bounds):
            if i == token_bounds[token_bounds_ind][0]:
                token_ind = token_bounds_ind

        if i == original_entities[entities_ind][0]:
            new_entities[entities_ind][0] = token_ind
        if i == original_entities[entities_ind][1] - 1:
            # -1 because entities store index of end like (end_index + 1) 
            new_entities[entities_ind][1] = token_ind
            new_entities[entities_ind][2] = original_entities[entities_ind][2]
            entities_ind += 1

        if token_bounds_ind < len(token_bounds):
            if i == token_bounds[token_bounds_ind][1]:
                token_bounds_ind += 1
                continue

        i += 1
    
    return new_entities

# example
text = "Hello, I'm a writer. Nice to meet you"
tokens = text_to_tokens_2(text)
entities = [[11, 19, "Profession"], [34, 37, "Person"]]
print(f"Text: {text}")
print(f"Tokens: {tokens}")
print(f"Entities: {entities}")
print(f"Result: {get_updated_entites_array(text, tokens, entities)}")

Text: Hello, I'm a writer. Nice to meet you
Tokens: ['Hello', ',', 'I', "'", 'm', 'a', 'writer', '.', 'Nice', 'to', 'meet', 'you']
Entities: [[11, 19, 'Profession'], [34, 37, 'Person']]
Result: [[5, 6, 'Profession'], [11, 11, 'Person']]


Функция ниже использует все предыдущие вспомогательные методы для получения массива токенов и массива сущностей для каждого токена согласно "BIO" разметке

In [16]:
# final functions to get tokens and their labels

def get_tokens_and_labels(text, entities):
    entities.sort(key=lambda x: x[0])
    
    text_tokens = text_to_tokens_2(text)

    # make new entities array with bounds relates to text_tokens
    new_entities = get_updated_entites_array(text, text_tokens, entities)
    
    # make labels
    labels = ['O' for _ in range(len(text_tokens))]
    for entity in new_entities:
        labels[entity[0]] = 'B-' + entity[2]
        for i in range(entity[0] + 1, entity[1] + 1):
            labels[i] = 'I-' + entity[2]

    return text_tokens, labels

### Обработка всех входных файлов и сохранение результата в один файл

In [18]:
# process all the new files and save result to a single file

new_df = pd.DataFrame(columns=['tokens', 'labels'])
for path in ALL_PATHS:
    temp_df = pd.read_json(path, lines=True)

    current_df = pd.DataFrame([['' for i in range(2)] for _ in range(temp_df.shape[0])], columns=['tokens', 'labels'])
    for i in range(temp_df.shape[0]):
        tokens, labels = get_tokens_and_labels(temp_df['text'][i].lower(), temp_df['entities'][i])

        current_df['tokens'][i] = tokens
        current_df['labels'][i] = labels
    
    new_df = pd.concat([new_df, current_df], axis=0, ignore_index=True)

new_df

,tokens,labels
0,"[276626, математические, основания, информатик...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,"[278248, компьютерная, графика, в, курсе, изуч...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,"[288500, информационная, безопасность, цифровы...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,"[282060, разработка, безопасного, программного...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,"[275487, автоматизация, функционально, -, логи...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
...,...,...
11047,"[название, дисциплины, :, системы, поддержки, ...","[O, O, O, B-Technology, I-Technology, I-Techno..."
11048,"[название, дисциплины, :, электромагнитные, по...","[O, O, O, B-Technology, I-Technology, I-Techno..."
11049,"[название, дисциплины, :, материалы, фотоники,...","[O, O, O, B-Technology, I-Technology, O, O, O,..."
11050,"[название, дисциплины, :, методологические, пр...","[O, O, O, B-Method, I-Method, B-Technology, I-..."


На моем вычислительном устройстве обработка заняла 1 мин и 11 сек

Проверим, какие получились сущности

In [ ]:
# check all possible tags
def get_all_entities(df):
    all_entities = set()
    for i in range(df.shape[0]):
        for j in range(len(df['labels'][i])):
            all_entities.add(df['labels'][i][j])
    return all_entities

print(get_all_entities(new_df))

{'I-SoftSkills', 'I-ProgrammingLanguage', 'B-Technology', 'I-Technology', 'I-Knowledge', 'I-Tool', 'B-Method', 'B-Knowledge', 'I-Method', 'B-ProgLanguage', 'O', 'B-ProgrammingLanguage', 'B-Tool', 'B-SoftSkills', 'I-ProgLanguage', 'B-'}


Заменим "ProgLanguage" на "ProgrammingLanguage" и удалим выброс в виде разметки "B-" 

In [ ]:
for i in range(new_df.shape[0]):
    for j in range(len(new_df['labels'][i])):
        if new_df['labels'][i][j] == 'B-ProgLanguage':
            new_df['labels'][i][j] = 'B-ProgrammingLanguage'
        elif new_df['labels'][i][j] == 'I-ProgLanguage':
            new_df['labels'][i][j] = 'I-ProgrammingLanguage'
        elif new_df['labels'][i][j] == 'B-':
            new_df['labels'][i][j] = 'O'

print(get_all_entities(new_df))

In [21]:
new_df.to_json('data_clean.json', index=False)

In [23]:
# check if the file is correct
df = pd.read_json('data_clean.json')
df.head(5)

,tokens,labels
0,"[276626, математические, основания, информатик...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,"[278248, компьютерная, графика, в, курсе, изуч...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,"[288500, информационная, безопасность, цифровы...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,"[282060, разработка, безопасного, программного...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,"[275487, автоматизация, функционально, -, логи...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
